In [8]:
from google.oauth2 import service_account

In [14]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=5eOCd84GLhGCnYZUedKXTqQbOhRijl&access_type=offline&code_challenge=J8tvEnw7sKNDrY3noX-o8rcSE5mT-m6UDqsD040ttDA&code_challenge_method=S256


Credentials saved to file: [/home/piraivn/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded" or "API not enabled" error. Run $ gcloud auth application-default set-quota-project to add a quota project.


In [18]:
from os.path import expanduser

HOME = expanduser("~")
credentials = service_account.Credentials.from_service_account_file(f"{HOME}/Downloads/aiserver_sa.json")

In [1]:
from langchain_google_vertexai import VertexAI

In [20]:
model = VertexAI(model_name="gemini-2.5-flash", credentials=credentials)

In [21]:
model.invoke("Hello, how are you?")

"Hello! I'm doing well, thank you for asking.\n\nHow can I help you today?"

In [33]:
from langchain_community.utilities import GooglePlacesAPIWrapper

In [ ]:
import os
os.environ["GPLACES_API_KEY"] = os.getenv("GPLACES_API_KEY")
places = GooglePlacesAPIWrapper()

In [38]:
places.run("tourist attraction")

"1. Vijinapura Lake\nAddress: 89, Nagappa Reddy Layout, Dooravani Nagar, Bengaluru, Karnataka 560016, India\nGoogle place ID: ChIJ9aRU_oYRrjsRXCWw3TRhVZA\nPhone: 080 2235 2828\nWebsite: Unknown\n\n\n2. Jinkethimmanahalli Lake\nAddress: 2MGP+MRV, TC Palya, Battarahalli, Bengaluru, Karnataka 560036, India\nGoogle place ID: ChIJj651qYYRrjsR9SQOiCe-d2U\nPhone: Unknown\nWebsite: Unknown\n\n\n3. Model Village\nAddress: Rachenahalli, Thanisandra, Bengaluru, Karnataka 560092, India\nGoogle place ID: ChIJS1IR7MEZrjsRcQy4o6CgpUk\nPhone: 080 2223 0060\nWebsite: https://mgired.karnataka.gov.in/page/Model+Heritage+Village+(Rangoli+Gardens)/en\n\n\n4. Kundalahalli Lake\nAddress: C-33-1, Kundalahalli, Whitefield, Bengaluru, Karnataka 560037, India\nGoogle place ID: ChIJiYFohwQTrjsRCXp2APahZes\nPhone: Unknown\nWebsite: Unknown\n\n\n5. Watch Tower - sunset and lake view tower\nAddress: Sri Venkateshpura Layout, Sampigehalli, Bengaluru, Karnataka 560064, India\nGoogle place ID: ChIJS9Ey7rAZrjsR9TTiy-axm

In [ ]:
from langchain.tools import tool
import requests

GOOGLE_API_KEY = os.environ["GPLACES_API_KEY"]
PLACES_ENDPOINT = "https://maps.googleapis.com/maps/api/place/textsearch/json"

def gather_places(destination: str, days: int):
    """
    Gathers popular places in and around the destination.
    Returns a list of {name, location_link, google_review_rating, other_link}.
    """
    results = []

    # 1. Google Places API
    params = {
        "query": f"stays near {destination}",
        "key": GOOGLE_API_KEY
    }
    resp = requests.get(PLACES_ENDPOINT, params=params).json()
    
    for place in resp.get("results", []):
        results.append(place)

    # 2. Blog/News search (example: Tavily API / SerpAPI)
    # You’d fetch extra sources & update other_link

    # 3. Adjust for trip days → filter/sort
    if days <= 2:
        results = results[:5]  # nearby highlights
    elif days <= 5:
        results = results[:10]
    else:
        results = results[:20]  # include long-distance places

    return results


In [53]:
gather_places("nagarhole tiger safari", 5)

[{'business_status': 'OPERATIONAL',
  'formatted_address': 'Churicad, K Badaga, Ponnampet, Kutta, Karnataka 571250, India',
  'geometry': {'location': {'lat': 12.0115884, 'lng': 76.05860609999999},
   'viewport': {'northeast': {'lat': 12.01290517989272,
     'lng': 76.06000997989273},
    'southwest': {'lat': 12.01020552010728, 'lng': 76.05731032010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'icon_background_color': '#909CE1',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
  'name': 'Machaan Wilderness Lodge Nagarahole',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 2242,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105813888601137423887">Machaan Wilderness Lodge Nagarahole</a>'],
    'photo_reference': 'AciIO2dHI9P4jxYXfxkU6j4-nHc9K45U36iwzTk85ZtIQ5Hl2yo5DPBg2frgXkuKnyUYUQjk3T-aWhObNciH7spvizdyjlZT8z_DPi-aFCVZ5x6idvIjx--NPaOr_SJPYxQzlfojmZTrz